Ejercicio:
5. (1.5 puntos) IImplemente una simulación para el problema del robot aspiradora presentado en clase. La simulación debe cumplir las siguientes especificaciones:
El agente aspiradora puede percibir las hojas en cualquiera de las 8 casillas adyacentes (se incluyen las diagonales).
El agente puede percibir los muros del tablero
El agente aspiradora dispondrá de 10 acciones: moverse a cualquiera de las 8 casillas adyacentes aspirar o no realizar ningúna acción.
El agente obtendrá un punto por cada cuadrado que se límpie.
La posición inicial de la aspiradora deberá generarse de manera aleatoria en el tablero.
La aspiradora solo tiene energía para realizar 200 acciones. La acciones de aspirar y percipir las hojas no consumen energía.
El tablero debe tener un tamaño de 10 x 10.
Se deben ubicar 50 hojas de manera aleatoria en el tablero.

In [1]:
import random
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets

In [9]:
# -------------------------------------------------
# 1. Clase Tablero
# -------------------------------------------------
class Tablero:
    def __init__(self, filas=10, cols=10, tam_celda=(50, 50)):
        """Crea un tablero de filas x cols con celdas de tamaño tam_celda."""
       
        self.filas = filas                 # Cantidad de filas en el tablero
        self.cols = cols                   # Cantidad de columnas en el tablero
        self.tam_celda = tam_celda         # Tamaño de cada celda, (ancho, alto)
        self.out = widgets.HTML()          # Widget HTML para mostrar la tabla
        display(self.out)                  # Mostramos el widget en la celda

    def dibujar(self, objetos):
        """Dibuja los objetos en el tablero."""
        html = "<table border='1' style='border-collapse:collapse;'>"  # Inicia tabla
        for y in range(self.filas):                 # Recorremos cada fila
            html += "<tr>"                          # Inicia una fila en HTML
            for x in range(self.cols):             # Recorremos cada columna
                contenido = ""                     # Texto inicial vacío
                # Buscamos si hay algun objeto en (x, y)
                for obj in objetos:
                    if obj.x == x and obj.y == y:
                        # Si coincide, creamos un <div> con su símbolo,
                        # tamaño y ángulo.
                        contenido = (
                            f"<div style='font-size:{obj.size}px;"
                            f"transform: rotate({obj.angle}deg);'>"
                            f"{obj.char}</div>"
                        )
                # Agregamos la celda con el contenido (vacío u objeto)
                html += (f"<td style='width:{self.tam_celda[0]}px;"
                         f"height:{self.tam_celda[1]}px;'>"
                         f"{contenido}</td>")
            html += "</tr>"      # Cerramos la fila
        html += "</table>"        # Cerramos la tabla
        self.out.value = html     # Actualizamos el widget con el HTML resultante

In [7]:
# -------------------------------------------------
# 2. Clases de Objetos
# -------------------------------------------------
class Objeto:
    """Objeto base que vive en una posición (x, y) con un caracter y un ángulo."""
    def __init__(self, x, y, char="?", angle=0, size=30):
        self.x = x
        self.y = y
        self.char = char
        self.angle = angle
        self.size = size

class Hoja(Objeto):
    def __init__(self, x, y):
        super().__init__(x, y, char="🍂", angle=0, size=30)

In [8]:
# -------------------------------------------------
# 3. Agente Reflejo
# -------------------------------------------------
class Aspiradora(Objeto):
    """
    Agente que se mueve en 8 direcciones, detecta paredes, detecta hojas.
    - 200 de energía para moverse.
    - Aspirar y percibir no consumen energía.
    - Gana 1 punto por cada hoja limpiada.
    """
    def __init__(self, x, y, energia=200):
        super().__init__(x, y, char="🤖", angle=0, size=35)
        self.energia = energia
        self.puntaje = 0

    def hay_hoja(self, hojas):
        """Si hay hoja en (x,y), la remueve y sube puntaje."""
        for h in hojas:
            if h.x == self.x and h.y == self.y:
                hojas.remove(h)
                self.puntaje += 1
                return True
        return False

    def mover(self, dx, dy, max_x=10, max_y=10):
        """
        Mueve al agente en (dx, dy) si no sale de 0..max-1. Cada movimiento consume 1 energía.
        Devuelve True si logró moverse, False si chocó en un muro.
        """
        nx = self.x + dx
        ny = self.y + dy
        if 0 <= nx < max_x and 0 <= ny < max_y:
            self.x = nx
            self.y = ny
            self.energia -= 1
            return True
        else:
            return False  # se chocó con un muro

    def percibir_8dirs(self, hojas, max_x=10, max_y=10):
        """
        Retorna qué hojas hay en las 8 celdas adyacentes, y si hay muro.
        moves_8 = [(-1, -1), (0, -1), (1, -1), (1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0)]
        """
        percepcion_hojas = []
        percepcion_muros = []
        moves_8 = [(-1, -1), (0, -1), (1, -1), (1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0)]
        for (dx, dy) in moves_8:
            nx = self.x + dx
            ny = self.y + dy
            # Checar si se sale del tablero
            if not (0 <= nx < max_x and 0 <= ny < max_y):
                percepcion_muros.append((dx, dy))
            else:
                # Revisar si hay hoja en (nx, ny)
                for h in hojas:
                    if h.x == nx and h.y == ny:
                        percepcion_hojas.append((dx, dy))
        return percepcion_hojas, percepcion_muros

    def pensar_y_actuar(self, hojas, max_x=10, max_y=10):
        """
        Comportamiento reflejo:
          1. Aspirar si hay hoja en la posicion actual.
          2. Percibir en 8 direcciones.
          3. Si ve una hoja adyacente, moverse hacia allí. Si se choca contra un muro, moverse random.
          4. Si no ve hoja, moverse random. (10ma accion puede ser 'no hacer nada' si se prefiere)
        """
        # Aspirar
        self.hay_hoja(hojas)

        # Si no queda energía, no hacemos nada
        if self.energia <= 0:
            return

        # Percibir 8 dirs
        percep_hojas, percep_muros = self.percibir_8dirs(hojas, max_x, max_y)

        # Accion
        dx, dy = 0, 0

        # Si percibo una hoja, mover en esa direccion
        if len(percep_hojas) > 0:
            # Elijo la primera hoja detectada
            (dx, dy) = percep_hojas[0]
            # Intentar moverse en (dx, dy)
            exito = self.mover(dx, dy, max_x, max_y)
            if not exito:
                # choca, moverse random
                self.movimiento_random(max_x, max_y)
        else:
            # no veo hojas, me muevo random
            self.movimiento_random(max_x, max_y)

        # Aspirar de nuevo si se paro sobre hoja
        self.hay_hoja(hojas)

    def movimiento_random(self, max_x=10, max_y=10):
        """Realizar un movimiento aleatorio en una de las 8 direcciones (si cabe). Si no cabe, no hacer nada."""
        moves_8 = [(-1, -1), (0, -1), (1, -1), (1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0)]
        random.shuffle(moves_8)
        for (dx, dy) in moves_8:
            if self.mover(dx, dy, max_x, max_y):
                return  # con un movimiento random que funcione, terminamos
        # si no pudo ninguno, la 10ma accion es no hacer nada (quedarse quieto)

In [11]:
# -------------------------------------------------
# 4. Generacion de Hojas y Aspiradora Aleatoria
# -------------------------------------------------
def generar_hojas_aleatorias(num=50, max_x=10, max_y=10):
    hojas = []
    used = set()
    while len(hojas) < num:
        x = random.randint(0, max_x-1)
        y = random.randint(0, max_y-1)
        if (x, y) not in used:
            used.add((x, y))
            hojas.append(Hoja(x, y))
    return hojas

def crear_aspiradora_aleatoria(max_x=10, max_y=10, energia=200):
    x = random.randint(0, max_x-1)
    y = random.randint(0, max_y-1)
    return Aspiradora(x, y, energia)

In [12]:
# -------------------------------------------------
# 5. Simulacion
# -------------------------------------------------
def simular_una_vez(tablero, max_acciones=300):
    # Generar 50 hojas
    hojas = generar_hojas_aleatorias(50, tablero.cols, tablero.filas)
    # Crear aspiradora
    asp = crear_aspiradora_aleatoria(tablero.cols, tablero.filas, energia=200)

    for _ in range(max_acciones):
        # Dibujar
        objetos = hojas + [asp]
        tablero.dibujar(objetos)
        time.sleep(0.05)  # para ver el movimiento

        # Si no hay mas energia, salimos
        if asp.energia <= 0:
            break

        # Si no hay mas hojas, salimos
        if len(hojas) == 0:
            break

        # Agente actua (reflejo)
        asp.pensar_y_actuar(hojas, tablero.cols, tablero.filas)

    # Ultima vista
    objetos = hojas + [asp]
    tablero.dibujar(objetos)

    return asp.puntaje

In [ ]:
# -------------------------------------------------
# 6. Pruebas y Graficar Resultados
# -------------------------------------------------
def ejecutar_varias_simulaciones(n=50):
    resultados = []
    for i in range(n):
        print(f"Simulacion {i+1}/{n}")
        tab = Tablero(10, 10)
        puntaje = simular_una_vez(tab)
        resultados.append(puntaje)
        print(f"Puntaje: {puntaje}")
        time.sleep(1)  # pausa entre simulaciones
    return resultados

def main():
    """ Ejecuta 50 simulaciones y muestra histograma."""
    res = ejecutar_varias_simulaciones()  # para no demorar, probamos 3
    plt.hist(res, bins=range(0, 52), edgecolor='black', align='left')
    plt.title("Histograma de hojas limpiadas")
    plt.xlabel("Hojas limpiadas")
    plt.ylabel("Frecuencia")
    plt.show()

main()

Estrategia basada en estado:

- El agente define un plan de barrido total en zigzag (crear_plan_zigzag), que recorre todas las celdas.
- Cada vez, se mueve un paso hacia su siguiente objetivo.
- Si se choca con un muro, elimina su plan (plan=None) de modo que su siguiente accion se reasigne random.
  (Esto es un simple ejemplo de "manejo de muros"; se puede diseñar otra estrategia como 'dar la vuelta'.)
- Aspiradora aspira la casilla actual sin gastar energía.
- 50 simulaciones, 200 energía, 50 hojas, 10x10.
- Se muestra histograma final.

¿Es posible un agente racional basado en estado?

Sí. Con este plan, cubre el tablero sin saltarse celdas y consigue limpiar la mayoría (o todas) las hojas,
siempre que la energía sea suficiente.

